# Linear Regression (Project 6)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
import numpy as np

In [ ]:
data = pd.read_csv("RRCA_baseflow.csv")
data

#### Removing rows if they have NaN values

In [ ]:
data = data.dropna()
data

#### It does not look to have any NaN values

### Adding new date features like year, months, and seasons

In [ ]:
##### Subtracting 693963 from Data column to make it from 01-01-1900
data["Date"] = data["Date"] - 693963
data['Date'] = pd.to_datetime(data.Date, unit='D', origin=pd.Timestamp('1900-01-01'))

data['year'] = data.Date.dt.year
data['month'] = data.Date.dt.month
data['day'] = data.Date.dt.day

display(data)

### We will use day and month for Defining seasons

### Defining seasons as well

In [ ]:
def get_season(month, day):
    if (month == 3 and day >= 20) or (month == 4) or (month == 5) or (month == 6 and day < 21):
        return 'Spring'
    elif (month == 6 and day >= 21) or (month == 7) or (month == 8) or (month == 9 and day < 23):
        return 'Summer'
    elif (month == 9 and day >= 23) or (month == 10) or (month == 11) or (month == 12 and day < 21):
        return 'Fall'
    else:
        return 'Winter'

data['season'] = data.apply(lambda row: get_season(row['month'], row['day']), axis=1)

data

## Applying One-Hot-Encoding for using in making the model


##### Also, to make the model simpler we just use `season`. Therefore, we remove `year, month, day, and Date` columns

#### We removed month and day because we used them in creating the new column named `season`

#### This part is for creating our model!

In [ ]:
encoder = OneHotEncoder()

OneHot = encoder.fit_transform(data[["Segment_id", "season"]])
one_hot_feature_names = encoder.get_feature_names_out(['Segment_id', "season"])
one_hot_df = pd.DataFrame(OneHot.toarray(), columns=one_hot_feature_names)
data_encoded = pd.concat([data, one_hot_df], axis=1)
data_for_models = data_encoded.drop(columns=["Segment_id", "Date", "year", "day", "season", "month"])
data_for_models

### Mean Baseflow over seasons

In [ ]:
barData = data.groupby(["season"]).agg({"Observed": "mean"}).reset_index()
sns.barplot(data=barData, x="season", y="Observed", order=["Spring", "Summer", "Fall", "Winter"])
plt.xlabel("Seasons")
plt.ylabel("Baseflow")
plt.title("Baseflow in different seasons")

##### Looks summer in average was the most dry one

### Precipitation vs Irrigation pumping

In [ ]:
# fig, axs = plt.subplots(1, 2, figsize=(12, 3))
precipitationData = data.groupby(["year"]).agg({"Precipitation": "sum", "Irrigation_pumping": "sum"}).reset_index()
sns.lineplot(y="Precipitation", x="year", data=precipitationData)
plt.title("Sum of Precipitation overs years")
plt.show()
sns.lineplot(y="Irrigation_pumping", x="year", data=precipitationData)
plt.title("Sum of Irrigation pumping overs years")
plt.show()


#### Looks Irrigation increased when Precipitation decreased over years

### Location of Baseflow observed

In [ ]:
sns.scatterplot(x="x", y="y", data=data, s=data["Observed"])
plt.xlabel("X")
plt.ylabel("y", rotation=60)
plt.title("Location") 
top_observed = data.groupby(['x', 'y']).apply(lambda x: x.nlargest(1, 'Observed')).reset_index(drop=True)

# Sort by 'Observed' in descending order to get the top 2 different (x, y) coordinates with the highest values in 'Observed'
top_2_observed = top_observed.nlargest(2, 'Observed')
display(top_2_observed)


### The top 2 locations were observed with their data info

## Finding the unique values of the segments

In [ ]:
segments = data["Segment_id"].unique()
segments

#### Baseflow in different segment_id over years

In [ ]:
for seg in segments:
    plt.scatter(x=data[data.Segment_id == seg].year, y=data[data.Segment_id == seg].Observed)
# data.plot(kind="scatter", x="year", y="Observed")
plt.xlabel("Year")
plt.ylabel("Observed")
plt.title("Observed Baseflow over years")

In [ ]:
for seg in segments:
    plt.scatter(x=data[data.Segment_id == seg].Evapotranspiration, y=data[data.Segment_id == seg].Observed)
plt.xlabel("Evapotranspiration")
plt.ylabel("Observed")
plt.title("Evapotranspiration vs Observed baseflow")    

##### It does not give us a good view so we have to test it in different segments

In [ ]:
for seg in segments:
    plt.scatter(x=data[data.Segment_id == seg].Precipitation, y=data[data.Segment_id == seg].Observed)
plt.xlabel("Precipitation")
plt.ylabel("Observed")
plt.title("Precipitation vs Observed baseflow")

#### We need to test this one in different segments as well because it does not give us valuable information

#### Applying regression line on chart

In [ ]:
for seg in segments:
    plt.scatter(x=data[data.Segment_id == seg].Irrigation_pumping, y=data[data.Segment_id == seg].Observed)
    
X = data_for_models[["Irrigation_pumping"]]
y = data_for_models["Observed"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2
)
lgModel = LinearRegression().fit(X_train, y_train)
preds = lgModel.predict(X_test)
plt.plot(X_test, preds, c='black', linewidth=3)
plt.xlabel("Irrigation_pumping")
plt.ylabel("Observed")
plt.title("Irrigation_pumping vs baseflow applied regression line")
plt.show()
print(f"Intercept: {lgModel.intercept_}")
print(f"Coef: {lgModel.coef_}")
print(f"R-Squared: {lgModel.score(X, y)}")

#### It looks that by increasing irrigation pumping the baseflow decreses but to make sure we will test it in different segments
#### And only this feature is not suitable for creating the model and we need to use other features as well.

## Irrigation pumping over time

In [ ]:
for seg in segments:
    plt.scatter(x=data[data.Segment_id == seg].year, y=data[data.Segment_id == seg].Irrigation_pumping)
plt.xlabel("Year")
plt.ylabel("Irrigation_pumping")
plt.title("Irrigation_pumping over years")

#### Looks pumping increased over the time

#### Seed defined to have the same result from selection in every runs

### Randomly selected segments to check out the correlations instead of checking all the segments

In [ ]:
random_seed = 5
random.seed(random_seed)
random_segments = random.sample(list(segments), 12)
print(random_segments)

#### Check if there is any correlation between randomly selected segments in Evapotranspiration vs observed baseflow

In [ ]:
fig, axs = plt.subplots(3, 4, figsize=(14, 8))
j = 0
k = 0
for seg in (random_segments):
  sns.regplot(x=data[data.Segment_id == seg].Evapotranspiration, y=data[data.Segment_id == seg].Observed, ax=axs[j, k], line_kws={"color": "red"})
  # axs[j, k].scatter(x=data[data.Segment_id == seg].Evapotranspiration, y=data[data.Segment_id == seg].Observed)
  axs[j, k].set_title(f"Segment {seg}", fontsize=15)
  axs[j, k].set_xlim(-1, 13)
  axs[j, k].set_ylim(-15, 150)
  k += 1
  if k == 4:
    j += 1
    k = 0
        
for ax in axs.flat:
  ax.set_xlabel("Evapotranspiration", fontsize=15)
  ax.set_ylabel("Baseflow", fontsize=16)

for ax in fig.get_axes():
  ax.label_outer()
fig.suptitle("Different segments Evapotranspiration vs observed baseflow", fontsize=16)
plt.show()

##### Sounds by increasing Evapotranspiration the baseflow decreases

#### Check if there is any correlation between randomly selected segments in Irrigation_pumping vs observed baseflow

In [ ]:
fig, axs = plt.subplots(3, 4, figsize=(14, 8), sharex=True, sharey=True)
j = 0
k = 0
for seg in (random_segments):
  sns.regplot(x=data[data.Segment_id == seg].Irrigation_pumping, y=data[data.Segment_id == seg].Observed, ax=axs[j, k], line_kws={"color": "red"})
    # axs[j, k].scatter(x=data[data.Segment_id == seg].Irrigation_pumping, y=data[data.Segment_id == seg].Observed)
  axs[j, k].set_title(f"Segment {seg}", fontsize=15)
    # axs[j, k].set_xlim(-5, 1)
  axs[j, k].set_ylim(-15, 150)
  k += 1
  if k == 4:
      j += 1
      k = 0
        
for ax in axs.flat:
  # ax.set(xlabel="Irrigation_pumping", ylabel="observed baseflow")
  ax.set_xlabel("Irrigation_pumping", fontsize=15)
  ax.set_ylabel("Baseflow", fontsize=16)

for ax in fig.get_axes():
  ax.label_outer()
fig.suptitle("Different segments Irrigation_pumping vs observed baseflow", fontsize=16)
plt.show()

#### Sounds by increasing (negative value means pumping out here means increasing pumping) irrigation pumping the basefolw decreases

#### Check if there is any correlation between randomly selected segments in Precipitation vs observed baseflow

In [ ]:
fig, axs = plt.subplots(3, 4, figsize=(14, 8), sharex=True, sharey=True)
j = 0
k = 0
for seg in (random_segments):
  sns.regplot(x=data[data.Segment_id == seg].Precipitation, y=data[data.Segment_id == seg].Observed, ax=axs[j, k], line_kws={"color": "red"})
  # axs[j, k].scatter(x=data[data.Segment_id == seg].Precipitation, y=data[data.Segment_id == seg].Observed)
  axs[j, k].set_title(f"Segment {seg}", fontsize=14)
  axs[j, k].set_ylim(-15, 100)
  k += 1
  if k == 4:
      j += 1
      k = 0
        
for ax in axs.flat:
  # ax.set(xlabel="Precipitation", ylabel="observed baseflow")
  ax.set_xlabel("Precipitation", fontsize=15)
  ax.set_ylabel("Baseflow", fontsize=16)

for ax in fig.get_axes():
  ax.label_outer()
fig.suptitle("Different segments Precipitation vs observed baseflow", fontsize=16)
plt.show()

In [ ]:
data_for_models

## Creating Linear Regression model and applying 10-fold cross-validation with all features

In [ ]:
model = LinearRegression()

kf = KFold(n_splits=10, shuffle=True, random_state=42)

X = data_for_models.drop(columns=["Observed"])
y = data_for_models["Observed"]

intercepts = []
coefficients = []
mse_scores = []
r_squared = []
i = 1
print("--------------------------")
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)
    
    intercepts.append(model.intercept_)
    coefficients.append(model.coef_)
    r_squared.append(r2_score(y_test, y_pred))
    print(f"R-Squared in fold {i}: {r2_score(y_test, y_pred)}")
    i += 1


print("--------------------------")
intercepts = np.array(intercepts)
coefficients = np.array(coefficients)


print("Mean CV R-Squared:", np.mean(r_squared))
print("Mean CV MSE:", np.mean(mse_scores))


print("Mean Intercept:", np.mean(intercepts))
print("------------------------------------")
print("This is the mean of the coefficient of all the features")
print("Mean Coefficients:", np.mean(coefficients, axis=0))

### Using all the features for making the model (did not use Cross-Validation)

In [ ]:
init = 'Observed ~ '
columns = data_for_models.columns
for i in range(len(data_for_models.columns)):
    if data_for_models.columns[i] != "Observed":
        init += data_for_models.columns[i]
        if i != len(data_for_models.columns) - 1:
            init += " + "
lm = smf.ols(formula=init, data=data_for_models).fit()
display(f"R-squared: {lm.rsquared}")
display(lm.params)
display(lm.summary())